# 課題4 テキストデータ分析
## Q1

$n$-次元ベクトル空間における、任意の2つのベクトル、$ \vec{x}=(x_1,x_2,...,x_n)$、$\vec{y}=(y_1,y_2,...,y_n)$、の間のcos類似度 $cos(X,Y)$ は以下のように定義される。

$cos(X,Y) =  \frac{\vec{x}\cdot \vec{y}}{\|x\|_2\|y\|_2}=\frac{\Sigma^{n}_{i=1}{x_{i}y_{i}}}{\sqrt{\Sigma^{n}_{i=1}x_{i}^{2}}\sqrt{\Sigma^{n}_{i=1}y_{i}^{2}}}$

入力ベクトル $\vec{x}$, $\vec{x}$ をそれぞれ`NumPy` の配列として引数で受け取り、それらのベクトル間のcos類似度を計算して返す関数 `cos_sim` を完成させる問題。

In [100]:
import numpy as np

In [101]:
def cos_similar(vec1, vec2):
    return np.dot(vec1, vec2) / ((np.linalg.norm(vec1))*(np.linalg.norm(vec2)))

In [102]:
print(cos_similar(np.array([1,0]), np.array([-1,0])))
print(cos_similar(np.array([1,0]), np.array([1,0])))
print(cos_similar(np.array([1,0]), np.array([0,-1])))

-1.0
1.0
0.0


## Q2
"course_list.csv"を読み取り、各講義のベクトルを作成し、講義間のcos類似度を計算する。

In [103]:
import json
import csv
import numpy as np
import pandas as pd

### Q2.1 
"course_list.csv"ファイルを1行ずつ順番に読み込み、その各行を要素とするリストを作成して返す`course_list`関数を完成させる。作成されたリストは変数`courses`で受けとるものとする。

In [104]:
def course_list():
    courses = []
    course_data = pd.read_csv('course_list.csv', header=None)
    for i in range(course_data.shape[0]):
        courses.append(course_data.loc[i, 0])
    return courses

In [105]:
courses =course_list()

### Q2.2
"keyword_list"には先ほどのcourst_listのキーワードが一語抜き出されたものが含まれている。
"keyword_list.csv"ファイルを1行ずつ**順番に**読み込み、その各行を要素とするリストを作成して返す`vocab_list`関数を完成させよ。作成されたリストは変数`vocab`で受け取る。以降の処理では、リスト`vocab`のインデックスをその要素（単語）のIDとして扱う。リスト`vocab`は以降の処理における語彙となる。

In [106]:
def vocab_list():
    vocab = []
    key_data = pd.read_csv("keyword_list.csv", header=None)
    for i in range(key_data.shape[0]):
        vocab.append(key_data.loc[i, 0])
    return vocab
vocab = vocab_list()

In [107]:
print(len(vocab))
print(vocab.index('プログラミング'))

978
164


### Q2.3
リスト`courses`と`vocab`を引数で受け取り、単語のID（リスト`vocab`のその単語のインデックス）をキー、その単語のDF（Document Frequency)を値とする辞書を作成して返す`count_df`関数を作成する。作成された辞書は変数`df`で受け取る。この場合、ある単語のDF値はその単語を講義名に含む講義数に対応する。

In [129]:
def count_df(_courses, _vocab):
    id_list = []
    df_list = []
    for k in range(len(vocab)):
        df_counter = 0
        for i in range(len(courses)):
            if vocab[k] in courses[i]:
                df_counter += 1
        id_list.append(k)
        df_list.append(df_counter)
    
    df = df_list
    return df

df = count_df(courses, vocab)

In [132]:
print(len(df))
print(164, vocab[164], df[164])

978
164 プログラミング 23


### Q2.4.1
リスト`vocab`の各単語を次元とする講義ベクトルを考える。講義ベクトルの長さはリスト`vocab`の長さと等しく、リスト`vocab`のインデックス`i`の単語`vocab[i]`が講義名に含まれる時、講義ベクトルの`i`番目の要素は`1`、含まれなければ`0`とする。

リスト`courses`と`vocab`を引数で受け取り、リスト`courses`の各講義のベクトルを行、リスト`vocab`の各単語を列とした`NumPy`の行列を作成して返す`lec_word_matrix`関数を完成させよ。作成した講義-単語行列は、講義（行）の講義名に単語（列）が含まれていれば、その要素が1であるような行列である。

In [134]:
def lec_word_matrix(_courses, _vocab):
    mat = np.zeros((len(courses), len(vocab)))
    
    for i in range(len(courses)):
        for j in range(len(vocab)):
            if vocab[j] in courses[i]:
                mat[i][j] = 1
    return mat

In [135]:
np.sum(lec_word_matrix(courses, vocab))

17731.0

### Q2.4.2
Q2.4.1で作成した講義-単語行列の各要素を、その講義の講義名に単語が含まれるか否かの1or0ではなく、その講義の講義名に単語が何回含まれるか（TF: Term Frequency)で表した行列を作成して返す`lec_word_tf_matrix`関数を完成させよ。

In [136]:
def lec_word_tf_matrix(_courses, _vocab):
    tf_mat = np.zeros((len(courses), len(vocab)))
    
    for i in range(len(courses)):
        for j in range(len(vocab)):
            if vocab[j] in courses[i]:
                tf_mat[i][j] = courses[i].count(vocab[j])
    return tf_mat

### Q2.4.3
Q2.4.2で作成した講義-単語行列の各要素（講義`i`の単語`j`の$TF_{ij}$）にその単語のIDF値を掛けたTFIDF値を要素とする行列を作成して返す`lec_word_tfidf_matrix`関数を完成させよ。

ここで、講義`i`、単語`j`のTFIDF値は以下のように定義される。
```Python
TFIDF=TF_ij*log(すべての講義数/単語jのDF)=TF_ij*log(len(courses)/df[j])
```

In [138]:
def lec_word_tfidf_matrix(_courses, _vocab, _df):
    tfidf_mat = np.zeros((len(courses), len(vocab)))
    
    for i in range(len(courses)):
        for j in range(len(vocab)):
            if vocab[j] in courses[i]:
                tfidf_mat[i][j] = courses[i].count(vocab[j]) * np.log(len(courses) / df[j])
    return tfidf_mat
tfidf_matrix = lec_word_tfidf_matrix(courses, vocab, df)

In [139]:
np.sum(tfidf_matrix)

78877.29015665206

### Q2.5
Q2.4.3で作成した講義-単語行列を元に、入力の講義に対してcos類似度に基づいて他のすべての講義との類似度を計算し、類似する講義名をキー、その類似度を値とした辞書を返す以下の`find_similar_course`関数を完成させよ。

In [146]:
def find_similar_course(_target, _matrix, _courses):
    sim_dict = []
    
    for i in range(len(courses)):
        sim = cos_similar(_matrix[_target], _matrix[i])
        if (sim != 0) and (_target !=0)and(sim!=1):
            sim_ls = []
            sim_ls.append(i)
            sim_ls.append(sim)
            sim_dict.append(sim_ls)
    sim_d = {k: v for (k, v) in sim_dict}
        
    return sim_d

In [147]:
title="医学データの統計解析実習" # 入力の講義名

results=find_similar_course(courses.index(title), tfidf_matrix, courses) # 類似講義の辞書を受け取る

df=pd.DataFrame(list(results.items()), columns=['title', "similarity"]) # 類似度が上位の講義を表示
df.sort_values(by='similarity', ascending=False).head(10)

,title,similarity
119,2910,0.951426
86,2068,0.591201
85,2045,0.551596
105,2618,0.549322
166,3832,0.518601
71,1859,0.459255
57,1564,0.394401
18,222,0.392754
66,1745,0.385049
61,1680,0.385049
